# Requirements

In [25]:
# Add as many imports as you need.
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np

# Laboratory Exercise - Run Mode (8 points)

## Introduction
This laboratory assignment's primary objective is to fine-tune a pre-trained language model for binary classification on a dataset consisting of wine reviews. The dataset contains two attributes: **description** and **points**. The description is a brief text describing the wine and the points represent a quality metric ranging from 1 to 100. If some wine has at least 90 points it is considered **exceptional**. Your task involves predicting if some wine is **exceptional** based on its review.

## The Wine Reviews Dataset

Load the dataset using the `datasets` library.

In [2]:
# Write your code here. Add as many boxes as you need.
dataset = load_dataset('csv',data_files={'../datasets/wine-reviews.csv'})
dataset

DatasetDict({
    train: Dataset({
        features: ['description', 'points'],
        num_rows: 10000
    })
})

## Target Extraction
Extract the target **exceptional** for each wine review. If some wine has at least 90 points it is considered **exceptional**.

In [3]:
# Write your code here. Add as many boxes as you n
def modify_points(example):
    if example['points'] >=90:
        example['points'] = 1
    else:
        example['points'] = 0
    return example

In [4]:
dataset['train'] = dataset['train'].map(modify_points)

In [5]:
dataset['train'][:5]

{'description': ["Translucent in color, silky in the mouth, this Mendocino-grown Pinot shows ripe, forward flavors and crisp acidity. You'll find pie-filling cherries and rhubarb, with a cola and root beer soda finish.",
  'On the palate, this wine is rich and complex, with flavors of dried cherry, cola, licorice, red currant and sandalwood that finish bone dry. It has lots of acidity and some sandpaper-like tannins. This should begin to develop bottle-age notes after 2016.',
  "The producer blends 57% Chardonnay from the Maldonado Vineyard with 39% Sauvignon Blanc and 4% Viognier for this fresh, floral white. On the palate, apple and lemon flavors are buttressed by a core of laser-like acidity. A fresh pear note lasts long into the finish, proving the wine's ability to be both creamy and crisp.",
  "Pure Baga in all its glory, packed with dry and dark tannins and a brooding character that is mitigated by the rich dark plum and blackberry jelly flavors. It's a wine that is probably at 

## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.


In [6]:
# Write your code here. Add as many boxes as you need.
split_dataset = dataset['train'].train_test_split(test_size=0.2,seed=42)
split_dataset = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

## Tokenization
Tokenize the texts using the `AutoTokenizer` class.

In [7]:
# Write your code here. Add as many boxes as you need.
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [29]:
def preprocess_function(examples):
    tokenized = tokenizer(
        examples['description'], truncation=True, padding="max_length", max_length=15
    )
    tokenized['labels'] = examples['points']  # Ensure labels are included
    return tokenized


In [9]:
tokenized_dataset = split_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=['description']
)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['points', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['points', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

## Fine-tuning a Pre-trained Language Model for Classification
Fine-tune a pre-trained language model for classification on the given dataset.

Define the model using the `AutoModelForSequenceClassification` class.

In [11]:
# Write your code here. Add as many boxes as you need.
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the traning parameters using the `TrainingArguments` class.

In [12]:
# Write your code here. Add as many boxes as you need.
training_args = TrainingArguments(
    output_dir='./trainer',
    learning_rate=1e-5,
    metric_for_best_model='f1',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    evaluation_strategy='epoch',
    fp16=True
)

/home/david/miniconda3/envs/VNP/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Define the training using the `Trainer` class.

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [14]:
# Write your code here. Add as many boxes as you need.
trainer = Trainer(
    model=model,  # Ensure the model matches the task (e.g., binary classification)
    args=training_args,  # Training arguments
    train_dataset=tokenized_dataset['train'],  # Training dataset
    eval_dataset=tokenized_dataset['test'],  # Evaluation dataset
    processing_class=tokenizer,  # Tokenizer for compatibility
    compute_metrics=compute_metrics  # Metrics calculation function
)

In [15]:
import torch
torch.cuda.empty_cache()

Fine-tune (train) the pre-trained lanugage model.

In [16]:
# Write your code here. Add as many boxes as you need.
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.612400,0.600467,0.670500,0.671017,0.670500,0.670251


TrainOutput(global_step=1000, training_loss=0.6268985290527344, metrics={'train_runtime': 396.9863, 'train_samples_per_second': 20.152, 'train_steps_per_second': 2.519, 'total_flos': 31047046560000.0, 'train_loss': 0.6268985290527344, 'epoch': 1.0})

Use the trained model to make predictions for the test set.

In [23]:
# Write your code here. Add as many boxes as you need.
predictions = trainer.predict(tokenized_dataset['test'])

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [28]:
y_pred = np.argmax(predictions.predictions, axis=1)  # Convert logits to class predictions
y_true = np.array(tokenized_dataset['test']['points'])

# Generate the classification report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68      1000
           1       0.68      0.64      0.66      1000

    accuracy                           0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000



# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a simple machine learning pipeline to classify wine reviews as **exceptional** or not. Use TF-IDF vectorization to convert text into numerical features and train a logistic regression. Split the dataset into training and testing sets, fit the pipeline on the training data, and evaluate its performance using metrics such as precision, recall, and F1-score. Analyze the texts to find the most influential words or phrases associated with the **exceptional** wines. Use the coefficients from the logistic regression trained on TF-IDF features to identify the top positive and negative keywords for **exceptional** wines. Present these keywords in a simple table or visualization (e.g., bar chart).

In [ ]:
# Write your code here. Add as many boxes as you need.